In [ ]:
#from urllib.request import urlretrieve
#urlretrieve("http://pklab.med.harvard.edu/velocyto/hgForebrainGlut/hgForebrainGlut.loom", "hgForebrainGlut.loom")

# RNA velocity analysis with scVelo
##### RNA velocity analysis allows us to infer transcriptional dynamics that are not directly observed in a scRNA-seq experiment using a mathematical model of transcriptional kinetics. We can use RNA velocity to determine if a gene of interest is being induced or repressed in a give cell population of interest. Moreover, we can extrapolate this information to predict cell fate decision via pseudotime trajectories.

In [ ]:
import os
import shutil

dir = 'figures'
if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv

#scv.logging.print_version()
scv.settings.verbosity = 2  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualization
#jupyter nbconvert --no-input --to html Untitled.ipynb 
#papermill Untitled.ipynb mynotebook_output.ipynb

# 1. Standard pre-processing for scRNA-seq
##### This filters cells with low counts of spliced/unspliced mRNA molecules, and keeps the top 2000 highly variable genes.

In [ ]:
#variables
fieName=''
clusters=''
basis=""
"""
fieName='hgForebrainGlut.loom'
new_cluster_names=""
basis="tsne"
clusters='Clusters'
"""

In [ ]:
new_cluster_names=""
adata = scv.read(fieName)
adata.var_names_make_unique()

# standard pre-processing for scRNA-seq
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000) #this filters cells with low counts of spliced/unspliced mRNA molecules, and keeps the top 2000 highly variable genes
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

In [ ]:
## This section is for changing the nomenclature of the predefined numerical clusters in the original file
#new_cluster_names="Radial glia 1,Radial glia 2,Neuroblast,Immature Neuron 1,Immature Neuron 2,Neuron 1,Neuron 2"

if new_cluster_names!="":

    new_cluster_names=new_cluster_names.split(',')
    print(new_cluster_names)
    """new_cluster_names = ['Radial glia 1', 'Radial glia 2',
        'Neuroblast',
        'Immature Neuron 1', 'Immature Neuron 2',
        'Neuron 1', 'Neuron 2']"""

    adata.obs[clusters]=adata.obs[clusters].astype('category')
    adata.rename_categories(clusters, new_cluster_names)
    
newNames = np.array(adata.obs[clusters])
adata.obs['Cell_types'] = newNames


In [ ]:
## 1.1 Spliced/unspliced proprtions.

In [ ]:
#scv.pl.proportions(adata, groupby='Cell_types',save='spliced_unspliced_prop.png')

# 2. Estimate RNA velocity and latent time

In [ ]:
# Estimate RNA velocity and latent time
scv.tl.recover_dynamics(adata)
scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)
scv.tl.recover_latent_time(adata)
scv.tl.umap(adata)
scv.tl.tsne(adata)

# 3. Visualization
### 3.1 Visualize the velocities in UMAP embedding and color the cells by the cluster

In [ ]:
scv.pl.velocity_embedding(adata, arrow_length=3, basis=basis,color = 'Cell_types', legend_loc = 'on data', arrow_size=1.4,legend_fontsize=10,save='velocity_umap.png', dpi=150)

### 3.2 Visualize the velocities in UMAP embedding with latent time. The latent time represents the cell’s internal clock and approximates the real time experienced by cells as they differentiate.

In [ ]:
Colorss=["#E41A1C","#377EB8","#4DAF4A","#984EA3","#FF7F00","#FFFF33","#A65628","#F781BF"]
# Generate plot with UMAP and latent time
scv.pl.velocity_embedding_stream(adata,basis=basis,save='umap_latent_time.png',color="latent_time",fontsize=20,legend_fontsize=20,min_mass=2,color_map="plasma")

### 3.3 Visulize the latent time for each cluster using a violin plot.

In [ ]:
sc.pl.violin(adata, save="_prop.png",keys='latent_time',groupby="Cell_types")

# 4.Highly dynamic genes (HDG)
##### Rank genes for velocity characterizing groups. This applies a differential expression test (Welch t-test with overestimated variance to be conservative) on velocity expression, to find genes in a cluster that show dynamics that is transcriptionally regulated differently compared to all other clusters (e.g. induction in that cluster and homeostasis in remaining population). If no clusters are given, it priorly computes velocity clusters by applying louvain modularity on velocity expression.

In [ ]:
scv.tl.rank_velocity_genes(adata, groupby='Cell_types', min_corr=.3)
df = scv.DataFrame(adata.uns['rank_velocity_genes']['names'])
df.to_csv('./figures/Highly-dynamic-genes.csv')
df.head()

### 4.1 Scatter Plot: For top 5 of every cell type
##### Maximum 20 Genes

In [ ]:
geneList=[]
for column in df.head():
    geneList=geneList+df.head()[column].tolist()
    if len(geneList)>19:
        break
scv.pl.scatter(adata, basis=geneList,save="scatterPlot__HDG.png", ncols = 5, color = 'Cell_types', figsize = (14,14),frameon=False)

### 4.2 Heatmap
##### Expression of these top 5 genes (for every cell type) in each cell sorted with their latent time. 

In [ ]:
scv.pl.heatmap(adata, var_names=geneList, save="heatMap__HDG.png",sortby='latent_time', col_color='Cell_types', n_convolve=100,figsize=(16,8),yticklabels=True,sort=True,colorbar=True,show=True,layer="count")

# 5. Top-likelihood genes (TLG)
##### Driver genes display pronounced dynamic behavior and are systematically detected via their characterization by high likelihoods in the dynamic model.

In [ ]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:100]
df = adata.var['fit_likelihood'].sort_values(ascending=False)[:100]
df.to_csv('./figures/Top-likelihood-genes.csv')

df.head()

### 5.1 Scatter Plot: Top 20

In [ ]:
scv.pl.scatter(adata, basis=top_genes[:20], save="scatterPlot__TLG.png",ncols = 5, color = 'Cell_types', figsize = (14,14),frameon=False)

### 5.2 Heatmap
##### Expression of these top 20 genes in each cell sorted with their latent time. 

In [ ]:
scv.pl.heatmap(adata, var_names=top_genes[:20], save="heatmap__TLG.png",sortby='latent_time', col_color='Cell_types', n_convolve=100,figsize=(16,8),yticklabels=True,sort=True,colorbar=True,show=True,layer="count")